## Project Stage - V (Dashboard)  ddl: 04/28/2023

## Goals

The final stage aims a developing a simple interactive dashboard based on the analysis you have done so far. In this we will be utilizing Plotly (https://plotly.com/) along with Dash (https://plotly.com/dash/) as our framework. 

Refer here for Plotly: https://github.com/UNCG-CSE/CSC-405-605_Spring_2021/blob/master/Class_Resources/Lecture_10/Visualization/03_Plotly/Plotly.ipynb

Getting started with Dash: https://www.youtube.com/watch?v=hSPmj7mK6ng

*PS: This can be invoked from Jupyter, see here: https://medium.com/plotly/introducing-jupyterdash-811f1f57c02e*

### Tasks for stage V (team):

#### Task 1: (70 pts)

    - Main graph
        - Allow for selection of date to show the trend of COVID-19 cases and deaths. (30)
        - Allow for linear or log mode selection on the number of cases and deaths. (10)
        - Incorporate your best model prediction trend line - Linear / Non-Linear. (30)
        - Ex: https://ourworldindata.org/coronavirus
        
#### Task 2: (30 pts)

    - Trend
        - Plot the trend line using moving average (https://en.wikipedia.org/wiki/Moving_average). Use 7-day moving average. (15)
        - Allow for selection of multiple states on the same graph. (15)
    
    

**Deliverable**
- Take screenshots of Report upload on canvas.
- Each member creates separate notebooks for member tasks. Upload all notebooks to Github Repository. 
- Final Presentation recordings on canvas.

In [15]:
import pandas as pd

def get_us_confirmed_cases_time_series():
    df = pd.read_csv('../Stage IV/team_work/data/covid_confirmed_usafacts.csv')
    df = df.drop(['countyFIPS', 'County Name', 'State', 'StateFIPS'], axis=1)
    df = df.transpose()
    df.index = pd.to_datetime(df.index, format="%Y-%m-%d")
    df = df.sum(axis=1)
    df.name = 'cases'
    return df

def get_us_confirmed_deaths_time_series():
    df = pd.read_csv('../Stage IV/team_work/data/covid_deaths_usafacts.csv')
    df = df.drop(['countyFIPS', 'County Name', 'State', 'StateFIPS'], axis=1)
    df = df.transpose()
    df.index = pd.to_datetime(df.index, format="%Y-%m-%d")
    df = df.sum(axis=1)
    df.name = 'deaths'
    return df

df = pd.merge(get_us_confirmed_cases_time_series(), get_us_confirmed_deaths_time_series(), left_index=True, right_index=True)
df

,cases,deaths
2020-01-22,723,1
2020-01-23,734,1
2020-01-24,741,1
2020-01-25,751,1
2020-01-26,759,1
...,...,...
2023-01-12,96613336,1073572
2023-01-13,96622484,1073677
2023-01-14,96626030,1073692
2023-01-15,96627980,1073692
